In [1]:

import tensorflow as tf
from tensorflow import keras
import numpy as n

#123

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
    model=keras.Sequential([
      keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
    return model

In [9]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 2.6MB/s 


In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch( build_model, objective='val_accuracy', 
                     max_trials=5, directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 11m 22s]
val_accuracy: 0.8690000176429749

Best val_accuracy So Far: 0.9136666655540466
Total elapsed time: 00h 38m 12s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |48                
conv_1_kernel     |5                 |3                 
conv_2_filter     |64                |48                
conv_2_kernel     |3                 |5                 
dense_1_units     |128               |80                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 206s 122ms/step - loss: 0.3835 - accuracy: 0.8618 - val_loss: 0.2902 - val_accuracy: 0.8942
Epoch 2/3
1688/1688 [==============================] - 207s 123ms/step - loss: 0.2427 - accuracy: 0.9098 - val_loss: 0.2557 - val_accuracy: 0.9037
Epoch 3/3
 376/1688 [=====>........................] - ETA: 2:36 - loss: 0.1888 - accuracy: 0.9306

KeyboardInterrupt: ignored

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()